In [1]:
import sys
sys.path.append("../../smsl")

In [2]:
from smsl.graphBigtraj import BigTrajGraphAgent
from smsl.graph import CLST

import warnings; warnings.filterwarnings("ignore")
import numpy as np

In [3]:
## 見論文表B.7
n_mode=30         ## $\alpha_c$
vlv_threshold=0.1 ## $\nu_{\text{c}}$
pe_threshold=0.6  ## $\kappa_{\text{c}}$

In [4]:
b_agent = BigTrajGraphAgent(CLST)

Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/0000_0500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/0250_0750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/0500_1000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/0750_1250/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/1000_1500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/1250_1750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /ho

In [5]:
b_agent.spectral_decomposition()
b_agent.set_mean_r_alpha_array()
b_agent.t_agent.mean.GetEigenvectorEigenvectorT()

Thare are 599 nodes.
Initialize adjacency, degree and Laplacian matrices... Done.
Finish the setup for Laplaican matrix.
Thare are 599 nodes.
Initialize adjacency, degree and Laplacian matrices... Done.
Finish the setup for Laplaican matrix.
Thare are 599 nodes.
Initialize adjacency, degree and Laplacian matrices... Done.
Finish the setup for Laplaican matrix.
Thare are 599 nodes.
Initialize adjacency, degree and Laplacian matrices... Done.
Finish the setup for Laplaican matrix.
Thare are 599 nodes.
Initialize adjacency, degree and Laplacian matrices... Done.
Finish the setup for Laplaican matrix.
Thare are 599 nodes.
Initialize adjacency, degree and Laplacian matrices... Done.
Finish the setup for Laplaican matrix.
Thare are 599 nodes.
Initialize adjacency, degree and Laplacian matrices... Done.
Finish the setup for Laplaican matrix.
Thare are 599 nodes.
Initialize adjacency, degree and Laplacian matrices... Done.
Finish the setup for Laplaican matrix.
Thare are 599 nodes.
Initialize 

In [6]:
b_agent.plot_prominent_modes_and_get_identify_pair_with_interactive(\
        strandid=None, vlv_threshold=vlv_threshold, pe_threshold=pe_threshold, filter=True)

interactive(children=(IntSlider(value=1, description='sele_mode', max=599, min=1), Output()), _dom_classes=('w…

### 確認機械熱點原子對$\{\text{a}_i\text{a}_j\}^{\text{bm}}$的機械強度$k_{ij}$是否較高

In [7]:
df = b_agent.t_agent.mean.df_m
mask1 = df['Resid_i']==15
mask2 = df['Resid_j']==22
df = df[mask1&mask2]
df.sort_values(by='k', ascending=False)

,Enm_i,Strand_i,Resid_i,Atomname_i,Atomid_i,Enm_j,Strand_j,Resid_j,Atomname_j,Atomid_j,k,b0,Category
PairID,,,,,,,,,,,,,


### Create $k^{cl\text{st}}$ table

In [8]:
from smsl.graphBigtraj import BigTrajGraphAgent
from smsl.config import ConfAgent

In [9]:
import os
import pandas as pd
import numpy as np
from IPython.display import JSON

In [10]:
class HotSpotAgent(dict, ConfAgent):
    def __init__(self, MAgent):
        ConfAgent.__init__(self)
        self.end_ns = self.time_num
        self.framesperns = self.frame_num // self.time_num
        self.MAgent = MAgent
    def load_systems(self, system2offical_name):
        ori_path = os.getcwd()
        for system, system_offical_name in system2offical_name.items():
            os.chdir(f'../../{system}/d.fluctmatch')
            b_agent = BigTrajGraphAgent(self.MAgent)
            b_agent.system_offical_name = system_offical_name
            self[system] = b_agent
        os.chdir(ori_path)
    def get_hotspot(self, resid_ij2cluster, verbose=0, is_return=False):
        self.m_abbr = self.get_m_abbr() 
        dict_hotspot = {self.m_abbr:{}}
        for system, b_agent in self.items():
            dict_hotspot[b_agent.m_abbr][system] = {}
            dict_hotspot[b_agent.m_abbr][system] = get_hotspot_by_graph_agent(b_agent.t_agent, resid_ij2cluster=resid_ij2cluster, verbose=verbose)
        if is_return:
            return dict_hotspot
        else:
            self.dict_hotspot = dict_hotspot
    def get_m_abbr(self):
        return list(self.values())[0].m_abbr ## first system of sub group name
    def get_df_k_with_hotspots(self, k_value='k_avg_std', is_return=False): ## for s = clst
        global sort_system, sort_layer
        list_k_with_hotspots = []
        for s, dict_subgroup in self.dict_hotspot.items(): 
            for system, dict_system in dict_subgroup.items():
                for position, dict_position in dict_system.items():
                    pos_col, pos_row = position[0], position[1]
                    k_avg = dict_position['hotspot']['avg']['avg']
                    k_std = dict_position['hotspot']['avg']['std']
                    resid_i = dict_position['resid_i']
                    resid_j = dict_position['resid_j']
                    resn_i = self[system].strandid2sequence['STRAND1'][resid_i]
                    resn_j = self[system].strandid2sequence['STRAND1'][resid_j]
                    list_k_with_hotspots.append({
                        'system'   : self[system].system_offical_name,
                        'layer'    : pos_row,
                        'gstrand'  : pos_col,
                        'k_avg'    : k_avg,
                        'k_std'    : k_std,
                        'k_avg_std': f'{k_avg:.2f} ± {k_std:.2f}',
                        'resid_ij' : f'{resn_i}{resid_i}{resn_j}{resid_j}',
                    })
        df_k_with_hotspots_1 = pd.DataFrame(list_k_with_hotspots)
        df_k_with_hotspots = df_k_with_hotspots_1.set_index(['system', 'layer'])
        df_k_with_hotspots = df_k_with_hotspots.pivot(columns='gstrand', values=k_value)

        ## sort the row and column index
        fixed_layers  = ['topcap', 'topgroove', 'midgroove', 'botgroove', 'botcap']
        fixed_index   = pd.MultiIndex.from_product([list(system2offical_name.values()), fixed_layers], names=['system', 'layer'])
        fixed_columns = ['Q3', 'Q2', 'Q1', 'Q4']
        df_k_with_hotspots = df_k_with_hotspots.reindex(columns=fixed_columns, index=fixed_index).fillna('')
        if is_return:
            return df_k_with_hotspots
        else:
            self.df_k_with_hotspots = df_k_with_hotspots 

In [11]:
def generate_cluster_hotspots(self): ## for s = clst
    cluster2hotspots = {
        'core-loop-stack': {i:['rank',i] for i in range(1, 6+1)}, ## rank1..rank6
        'core-loop-hbond': {i:['rank',i] for i in range(1, 4+1)}, ## rank1..rank4
        # 'loop-loop-stack': {i:['rank',i] for i in range(1, 6+1)}, ## rank1..rank6
    }
    return cluster2hotspots

## SmallTimeAgent level
def get_cluster(self, resid_i, resid_j): ## for s = clst, i ∈ core, j ∈ loop
    pass


## SmallTimeAgent level
def generate_df_position2hotspots(self, resid_i, resid_j, hotspots): ## for s = clst
    dict_ks = {}
    is_switch = (resid_i > resid_j)
    if is_switch:
        resid_j, resid_i = resid_i, resid_j
    df_m = self.mean.df_m
    mask1 = (df_m['Resid_i']==resid_i) & (df_m['Resid_j']==resid_j)
    df_resid_ij = df_m[mask1]
    for i, hotspot in hotspots.items():
        if isinstance(hotspot[1], int): ## rank n: ['rank', 1]
            df_sele = df_resid_ij.sort_values(by='k', ascending=False).iloc[[hotspot[1]-1]]
        elif isinstance(hotspot[1], str): ## hotspot: ['N1', 'O6']
            if is_switch:
                mask2 = (df_resid_ij['Atomname_i']==hotspot[1]) & (df_resid_ij['Atomname_j']==hotspot[0])
            else:
                mask2 = (df_resid_ij['Atomname_i']==hotspot[0]) & (df_resid_ij['Atomname_j']==hotspot[1])
            df_sele = df_resid_ij[mask2]
        else:
            raise RuntimeError('Could not recognize cluster2hotspots:', cluster2hotspots)
        ## generate dict_ks
        pair_id, ser_sele = df_sele.index, df_sele.iloc[0]
        dict_ks[i] = {
            'query'     : ''.join(map(str, hotspot)),
            'atomname_i': ser_sele.Atomname_i if not is_switch else ser_sele.Atomname_j,
            'atomname_j': ser_sele.Atomname_j if not is_switch else ser_sele.Atomname_i,
            'k'         : {},
        }
        for time_label, st_agent in self.items():
            dict_ks[i]['k'][time_label] = st_agent.df_m.loc[pair_id].squeeze().k
        ks = pd.Series(dict_ks[i]['k'])
        dict_ks[i]['k']['avg'] = ks.mean()
        dict_ks[i]['k']['std'] = ks.std() ## Note: numpy.std(ddof=0), but pandas.std(ddof=1)
    ## k_avg in {aiaj}^s
    dict_ks['avg']  = {}
    for time_label, _ in self.items():
        ks = []
        for i, hotspot in hotspots.items():
            ks.append(dict_ks[i]['k'][time_label])
        dict_ks['avg'][time_label] = np.mean(ks)
    ks = pd.Series(dict_ks['avg'])
    dict_ks['avg']['avg'] = ks.mean()
    dict_ks['avg']['std'] = ks.std() ## Note: numpy.std(ddof=0), but pandas.std(ddof=1)
    return dict_ks

## GraphAgent level, for s = clst
def get_hotspot_by_graph_agent(self, resid_ij2cluster, verbose=0):
    dict_position = {}
    for (resid_i, resid_j), cluster in resid_ij2cluster.items():
        cluster2hotspots = generate_cluster_hotspots(self)
        hotspots = cluster2hotspots[cluster]
        if verbose:
            print(self.system, layer_i, gstrand_i, resid_i, resid_j, cluster, hotspots)
        dict_ks = generate_df_position2hotspots(self, resid_i, resid_j, hotspots)
        position = get_position(hs_agent, resid_i, resid_j, cluster)
        dict_position[position] = {
            'resid_i': int(resid_i), ## save json format from int istead of np.int64(cannot save) 
            'resid_j': int(resid_j),
            'cluster': cluster,
            'hotspot': dict_ks
        }
    return dict_position

def get_position(self, resid_i, resid_j, cluster):
    layer_i, gstrand_i = (self.df_tetrad_geometry == resid_i).stack()[lambda x: x].index[0]
    layer_pos = layer_i+'groove' if 'hbond' in cluster else layer_i+'cap'
    return (gstrand_i, layer_pos)

In [17]:
system2offical_name = {
# 'propeller': 'Propeller',
# 'basket'   : 'Basket',
# 'chair'    : 'Chair',
# 'hybrid-i' : 'Hybrid-I',
# 'hybrid-ii': 'Hybrid-II',
# 'dsdna'    : 'dsDNA',
'hybrid-ii_wttel26_poh': 'Hybrid-II-wtTel26-POH'
}

resid_ij2cluster = { ## 由上面graph得出的residue pair手動設定
# (22, 15): 'core-loop-stack', ## mode1
# ( 4,  3): 'core-loop-stack', ## mode2
# (12, 13): 'core-loop-stack', ## mode3
# ( 6,  8): 'core-loop-stack', ## mode4
# (16, 13): 'core-loop-stack', ## mode7
    
( 6,  8): 'core-loop-stack', ## mode1
(18,  9): 'core-loop-stack', ## mode2
(24, 25): 'core-loop-stack', ## mode3
}

In [18]:
hs_agent = HotSpotAgent(CLST)
hs_agent.load_systems(system2offical_name)
hs_agent.get_hotspot(resid_ij2cluster)

Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/0000_0500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/0250_0750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/0500_1000/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/0750_1250/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/1000_1500/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /home/user/workfolder/g4dna_protocol_demo/hybrid-ii_wttel26_poh/d.fluctmatch/bigtraj_fluctmatch/1250_1750/result/pairtypes_k_b0_cutoff_5.00.csv
Reading data from: /ho

In [19]:
JSON(hs_agent.dict_hotspot)

<IPython.core.display.JSON object>

In [20]:
hs_agent.get_df_k_with_hotspots(is_return=True)

gstrand                                   Q3 Q2           Q1           Q4
system                layer                                              
Hybrid-II-wtTel26-POH topcap                                             
                      topgroove                                          
                      midgroove                                          
                      botgroove                                          
                      botcap     3.52 ± 0.72     2.77 ± 1.90  2.40 ± 1.62

In [21]:
df_k_with_hotspots = \
    (hs_agent.get_df_k_with_hotspots(is_return=True)+\
    ' ('+\
    hs_agent.get_df_k_with_hotspots(k_value='resid_ij',is_return=True)+\
    ')').replace(' ()','')

df_k_with_hotspots

gstrand                                          Q3 Q2                   Q1  \
system                layer                                                   
Hybrid-II-wtTel26-POH topcap                                                  
                      topgroove                                               
                      midgroove                                               
                      botgroove                                               
                      botcap     3.52 ± 0.72 (G6T8)     2.77 ± 1.90 (G18A9)   

gstrand                                            Q4  
system                layer                            
Hybrid-II-wtTel26-POH topcap                           
                      topgroove                        
                      midgroove                        
                      botgroove                        
                      botcap     2.40 ± 1.62 (G24T25)